<a href="https://colab.research.google.com/github/Gaurav7004/RSE_TOOL/blob/main/RSE_TOOL_30thMarch2023_EVENING_6%3A00PM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
# ! pip install pandarallel
# from pandarallel import pandarallel
import json
import re

In [2]:
# pandarallel.initialize()

In [3]:
df = pd.read_csv('Data1.csv')

<ipython-input-3-7e89209b1ff8>:1: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Data1.csv')


In [4]:
## Data is the dictionary
data = {}

## Keys
key = ''

## values
values = []

## Open
with open('TEST_EXAMPLE_2.RSE', "r") as f:
    for line in f:
        line = line.strip()
        if line.startswith('#'):
            if key:
                key = key.split("'")[0]
                key = key.split("\t")[0]
                data[key] = [v for v in values if v and not v.startswith("'")]
            key = line.split("'")[0]
            key = key.split("\t")[0]
            values = []
        elif line.startswith("'"):
            pass
        else:
            line = line.split("'")[0]
            values.append(line.strip())

# add last group
if key:
    data[key] = [v for v in values if v and not v.startswith("'")]

print(json.dumps(data, indent=4))

{
    "#SAMPLING SUBSAMPLE2": [],
    "#GROUP 2": [
        "SEC   C sec",
        "ST_GR C NEWVAR"
    ],
    "#FILTER 1": [
        "AGE>=15"
    ],
    "#VARIABLE 3  ": [
        "POP\tN  NEWVAR",
        "LF\tN  NEWVAR",
        "WRK\tN  NEWVAR"
    ],
    "#FILE Data1.csv  ": [],
    "#RENAME  5    ": [
        "STRMID C sec+st+strm+sstrm",
        "SS     C ss",
        "NSS    N  nss",
        "NSC    N  nsc",
        "MULT   N  mult"
    ],
    "#TRANSFORM 4  ": [
        "AGE_Gr=(\"01\") AGE<5",
        "AGE_Gr=(\"02\") AGE>=5 && AGE<10",
        "AGE_Gr=(\"03\") AGE>=10 && AGE<20",
        "AGE_Gr=(\"04\") AGE>=20 && AGE<60",
        "AGE_Gr=(\"05\") AGE>=60",
        "YP=(1)",
        "YM=(1)        sex<>2",
        "YM=(0)\t\tOtherwise",
        "YF=(1)        sex=2",
        "YF=(0)\t\tOtherwise",
        "YH=(1)        GEDU>=\"11\"",
        "YH=(0)\t\tOtherwise",
        "newVar4=(1)  CWS in (\"11,12,21,31,41,51,61,61,71,72\")",
        "newVar4=(0)  CWS in (\"11,12,21,3

In [5]:
def SAMPLING_(data, df):
    for key, value in data.items():
        pass

def GROUP_(data, df):
    for key, value in data.items():
        ### To read the sampling method used
        if str('SAMPLING').lower() in str(key).lower():
            if str('SUBSAMPLE2').lower() in str(key).lower():
                return str('SUBSAMPLE2')
            elif str('SRSWR').lower() in str(key).lower():
                pass
            elif str('SRSWOR').lower() in str(key).lower():
                pass

def FILTER_(data, df):
    for key, value in data.items():
        ### To read the sampling method used
        if str('FILTER').lower() in str(key).lower():
            ### Replace logical AND '&&' with 'and'
            if '&&' in value[0]:
                return value[0].replace('&&', 'and')
            ### Replace logical OR '||' with 'or'
            elif '||' in value[0]:
                return value[0].replace('||', 'or')
            else:
                return value[0]

def FILE_(data):
    for key, value in data.items():
        ### To read the file used
        if str('FILE').lower() in str(key).lower():
            if key.strip().endswith('.xlsx') or key.strip().endswith('.csv'):
                match = re.search(r'[A-Za-z0-9]+\.xlsx|[A-Za-z0-9]+\.csv', key)   
                if match:
                    file_name = match.group(0)
                    return file_name
                else:
                    file_name = None
                    return file_name
                print(file_name)

            # return file_name

def RENAME_(data, df):
    # create an empty dictionary to store the column names
    column_dict = {}

    for key, value in data.items():
        ### To read the file used
        if str('RENAME').lower() in str(key).lower():
            # value = [elem for elem in value[0].split(' ') if elem.strip()]
            # return value

            for col in value:
                col_split = col.split()

                # print(col_split[0])

                # extract the last element as the column name
                col_name = col_split[-1]

                new_col_name = col_split[0]

                ### if there is a "+" sign in the last element, concatenate the columns
                if "+" in col_name:
                    col_concat = col_name.split("+")
                    col_concat = [i.upper() for i in col_concat]

                    # concatenate the columns
                    df[new_col_name] = df.loc[:, col_concat].apply(lambda x: ''.join(['0' + str(i) if i < 10 else str(i) for i in x]), axis=1)

                else:
                    col_name = col_name.upper()
                    print(type(col_name))


def NEWVARIABLE_(data, df):
    for key, value in data.items():
        ### To read the file used
        if str('VARIABLE').lower() in str(key).lower():
            value = [s.split()[0] for s in value]
            
            for name in value:
                df[name] = 0


def TRANSFORM_(data):
    # Initialize empty arrays to store the values
    Arr1 = []
    Arr2 = []
    Arr3 = []

    for key, value in data.items():
        ### To read the file used
        if str('TRANSFORM').lower() in str(key).lower():
            lines = '\n'.join(value)
            lines = lines.split("\n")

            # Loop through the lines and extract the values
            for line in lines:
                line_values = line.split('=')  # split the line into values
                
                var_name = line_values[0].strip()  # extract the variable name
                
                var_value = line_values[1].strip().split()[0]  # extract the variable value
                
                # Regular expression pattern to extract the condition
                pattern = r'^\w+=\([^\)]*\)(?:\s+(.*?)(?:\n|$))?$'

                # Split the string into lines and extract the condition from each line
                conditions = []

                for line in line.split('\n'):
                    match = re.match(pattern, line)
                    
                    if match:
                        condition = match.group(1)
                        if condition is not None:
                            conditions.append(condition.strip())
                        else:
                            conditions.append(None)

                # Append the values to the arrays
                Arr1.append(var_name)
                Arr2.append(var_value)
                Arr3.append(condition)

    return Arr1, Arr2, Arr3
        

def EST_RSE_(data):
    for key, value in data.items():
        pass

# print(FILE_(data))

TRANSFORM_(data)

# # print(FILE_(data))
# df = pd.read_csv(FILE_(data))
# df = df.rename(columns=lambda x: x.strip())

(['AGE_Gr',
  'AGE_Gr',
  'AGE_Gr',
  'AGE_Gr',
  'AGE_Gr',
  'YP',
  'YM',
  'YM',
  'YF',
  'YF',
  'YH',
  'YH',
  'newVar4',
  'newVar4'],
 ['("01")',
  '("02")',
  '("03")',
  '("04")',
  '("05")',
  '(1)',
  '(1)',
  '(0)',
  '(1)',
  '(0)',
  '(1)',
  '(0)',
  '(1)',
  '(0)'],
 ['AGE<5',
  'AGE>=5 && AGE<10',
  'AGE>=10 && AGE<20',
  'AGE>=20 && AGE<60',
  'AGE>=60',
  None,
  'sex<>2',
  'Otherwise',
  'sex=2',
  'Otherwise',
  'GEDU>="11"',
  'Otherwise',
  'CWS in ("11,12,21,31,41,51,61,61,71,72")',
  'CWS in ("11,12,21,31,41,51,61,61,71,72")'])

In [66]:
def parse_query_string(query_string):
    # Replace curly quotes with straight quotes
    query_string = query_string.replace("”", "\"")
    
    if 'in' in query_string:
        if ('NOT in' or 'not in') in query_string:
            query_string = re.sub(r'(\b\w+\b)\s+NOT in\s*\(([^)]+)\)', r'~df["\1"].isin([\2])', query_string)
            query_string = re.sub(r'(\b\w+\b)\s+in\s*\(([^)]+)\)', r'df["\1"].isin([\2])', query_string)
        elif 'in' in query_string:
            # Replace 'in' operators with 'isin' and add list brackets
            query_string = re.sub(r'(\b\w+\b)\s+in\s*\(([^)]+)\)', r'df["\1"].isin([\2])', query_string)
        else:
            pass
            
    else:
        query_string = re.sub(r'''(\b\w+\b)(\s*[<>=!]+\s*)('[^']*'|"[^"]*")|(\b\w+\b)(\s*[<>=!]+\s*)("[^]*'|"[^"]*")''', r'df["\1"]\2\3', query_string)
        query_string = re.sub(r'''(\b\w+\b)(\s*[<>=!]+\s*)(\d+)''', r'df["\1"]\2\3', query_string)

    # Replace double quotes with single quotes
    query_string = query_string.replace("\"", "'")
    
    # Replace 'AND' with '&', 'OR' with '|', and add parentheses
    query_string = query_string.replace(" AND ", " and ")\
                        .replace(" OR ", " or ").\
                            replace("&&", 'and').\
                                replace('||', 'or').\
                                    replace('<>', '!=')
    
    # Replace column names with df[column] syntax
    query_string = re.sub(r'(\b\w+\b)=', r'df["\1"]==', query_string)


    # Define a function to replace variable names with df["<variable_name>"]
    def replace_var(match):
        return f'df["{match.group(0)}"]'

    # Use regex to replace variable names with df["<variable_name>"]
    pattern = re.compile(r'''\b\w+\b(?=\s*[<>=!]=*\s*[\'"\d])''')
    new_string = pattern.sub(replace_var, query_string)

    query_string = re.sub(r"'(\d+)'|\"(\d+)\"", lambda match: str(int(match.group(1) or match.group(2))), query_string)
    
    return query_string

final_parsed_query = []

for i in list(TRANSFORM_(data))[2]:
    # print(i)
    if i:
        parsed_query = parse_query_string(i)
        final_parsed_query.append(parsed_query)
    else:
        final_parsed_query.append(i)

print(final_parsed_query)

["df['AGE']<5", "df['AGE']>=5 and df['AGE']<10", "df['AGE']>=10 and df['AGE']<20", "df['AGE']>=20 and df['AGE']<60", "df['AGE']>=60", None, "df['sex']!=2", 'Otherwise', "df['sex']=2", 'Otherwise', "df['GEDU']>=11", 'Otherwise', "df['CWS'].isin(['11,12,21,31,41,51,61,61,71,72'])", "df['CWS'].isin(['11,12,21,31,41,51,61,61,71,72'])"]


In [69]:
# # Define the input string
# input_str = '''ST_GR="G1"   in(ST,"10,19,20,21")
#                   ST_GR="G2"   in(ST,"11,12,13,14,15,16,17,18")
#                     ST_GR="G3"   Otherwise
#                         POP=1
#                             WRK=1  in(CWS,"11,12,21,31,41,51,61,61,71,72")
#                                 WRK=0  NOT in(CWS,"11,12,21,31,41,51,61,61,71,72")
#                                     LF=1      in(CWS,"11,12,21,31,41,51,61,61,71,72,81")
#                                         LF=0      Otherwise'''

# input_str = 'newVar4=1  CWS in (11,12,21,31,41,51,61,61,71,72)              * For persons Employed',\
#                 'newVar4=0  CWS not in (11,12,21,31,41,51,61,61,71,72)        * For rest',\
#                     'newVar5=1  CWS in (11,12,21,31,41,51,61,61,71,72,81)         * For persons in Labour Force',\
#                         'newVar5=0  CWS not in (11,12,21,31,41,51,61,61,71,72,81)   * For rest',\
#                             'newVar6=1  CWS in ()                                                                * For persons ALL',\
#                                 'newVar1=ERN_REG + ERN_SELF  GEDU_LVL in (05,06) – say for under graduates',\
#                             'newVar1=0 otherwise',\
#                         'newVar2=ERN_REG + ERN_SELF  GEDU_LVL in (07,08,09) – say for graduates',\
#                     'newVar2=0 otherwise',\
#                 'newVar3=ERN_REG + ERN_SELF  GEDU_LVL in (05,06,07,08,09) – addition of above two',\
#             'newVar3=0 otherwise'

# Split the input string into lines
# lines = input_str.split('\n')

# Initialize empty arrays to store the values
Arr1 = []
Arr2 = []
Arr3 = []

# Loop through the lines and extract the values
for line in final_parsed_query:
    print(line_values)
    line_values = line.split('=')  # split the line into values
    var_name = line_values[0].strip()  # extract the variable name
    print(var_name)
    var_value = line_values  # extract the variable value
    print(var_value)
    condition = 'None'  # initialize the condition to None

    # Check if the line contains "Otherwise"
    if "Otherwise" in line:
        condition = "Otherwise"

    # Check if the variable value contains an "in" statement
    if "in" in line:
        if "NOT in" in line:
            condition = line.split()[1] + ' ' + line.split()[2]
        else:
            condition = line.split()[1]

    # Append the values to the arrays
    Arr1.append(var_name)
    Arr2.append(var_value)
    Arr3.append(condition)

# Print the results
# print(Arr1)
# print(Arr2)
# print(Arr3)

Arr4 = []
Arr5 = []

for i in range(len(Arr3)):
    if 'in' in Arr3[i]:
        if 'NOT in' in Arr3[i]:
            # pattern
            pattern = r'^NOT\s+in\((\w+),\s*"([^"]+)"\)$'
            # match
            match = re.match(pattern, Arr3[i])

            Arr4.append(match.group(1))
            Arr5.append(match.group(2))
            
        else:
            # pattern
            pattern = r'^in\((\w+),\s*"([^"]+)"\)$'
            # match
            match = re.match(pattern, Arr3[i])

            Arr4.append(match.group(1))
            Arr5.append(match.group(2))

    elif 'Otherwise' in Arr3[i]:
        # pattern
        pattern = r'^Otherwise$'
        # match
        match = re.match(pattern, Arr3[i])

        Arr4.append(match.group(0))
        Arr5.append(None)

    else:
        Arr4.append(None)
        Arr5.append(None)


["df['AGE']<5"]
df['AGE']<5
["df['AGE']<5"]
["df['AGE']<5"]
df['AGE']>
["df['AGE']>", "5 and df['AGE']<10"]
["df['AGE']>", "5 and df['AGE']<10"]
df['AGE']>
["df['AGE']>", "10 and df['AGE']<20"]
["df['AGE']>", "10 and df['AGE']<20"]
df['AGE']>
["df['AGE']>", "20 and df['AGE']<60"]
["df['AGE']>", "20 and df['AGE']<60"]
df['AGE']>
["df['AGE']>", '60']
["df['AGE']>", '60']


AttributeError: ignored

In [9]:

lst = []
for value in Arr5:
    if value is None:
        lst.append(value)
    else:
        lst.append([int(x) for x in value.split(',')])

Arr5 = lst

In [10]:
set(Arr1) - set(list(df.columns))

{'newVar1', 'newVar2', 'newVar3', 'newVar4', 'newVar5', 'newVar6'}

In [11]:

# Check for name mismatches and create new columns
mismatched_names = set(Arr1) - set(df.columns)

new_columns = {name: [0] * df.shape[0] for name in mismatched_names}
df = df.assign(**new_columns)

mismatched_names

{'newVar1', 'newVar2', 'newVar3', 'newVar4', 'newVar5', 'newVar6'}

In [12]:
list(mismatched_names)

['newVar1', 'newVar4', 'newVar3', 'newVar6', 'newVar2', 'newVar5']

In [13]:
Arr5, Arr4, Arr3, Arr2, Arr1, df.columns

([None, None, None, None, None, None, None, None, None, None, None],
 [None, None, None, None, None, None, None, None, None, None, None],
 ['CWS', 'CWS', 'CWS', 'CWS', 'CWS', '+', 'None', '+', 'None', '+', 'None'],
 ['1', '0', '1', '0', '1', 'ERN_REG', '0', 'ERN_REG', '0', 'ERN_REG', '0'],
 ['newVar4',
  'newVar4',
  'newVar5',
  'newVar5',
  'newVar6',
  'newVar1',
  'newVar1',
  'newVar2',
  'newVar2',
  'newVar3',
  'newVar3'],
 Index(['SEC', ' ST', 'FSU', ' IDNO', ' SEG', ' SSS', ' SSU', ' STRM',
        '  SSTRM', '  SS', ' NSS', ' NSC', ' MULT', ' HH_SIZE', '  HHTYPE',
        '  RELG', '  SG', ' MHCE', '  DC_ALL', '  DC_ST', '  SRL', ' REL',
        ' SEX', ' MSEX', '  AGE', ' MARST', '  GEDU_LVL', '   PAS', ' PS_SS',
        '  CWS', '  ERN_SELF', '   ERN_REG', 'newVar1', 'newVar4', 'newVar3',
        'newVar6', 'newVar2', 'newVar5'],
       dtype='object'))

In [14]:
for i in range(len(Arr3)):

    if 'in' in Arr3[i]:
        curr_column = 0
        lst_stack = []

        ### For NOT in case 
        if 'NOT in' in Arr3[i]:
            if Arr1[i] in list(df.columns):
                df.loc[~df[Arr4[i]].isin(Arr5[i]), (Arr1[i])] = Arr2[i]
            else:
                pass

        ### For in case
        else:
            if Arr1[i] in list(df.columns) and Arr5[i] is not None:

                if 'or' or 'and' in Arr3[i]:
                    pass
                else:
                    # print(Arr1[i], Arr2[i], Arr4[i], Arr5[i])
                    print(i)
                    df.loc[df[Arr4[i]].isin(Arr5[i]), (Arr1[i])] = Arr2[i]
                    lst_stack.extend(Arr5[i])
                    curr_column = Arr4[i]



            else:
                pass

    elif 'Otherwise' in Arr3[i]:
            if Arr1[i] in list(df.columns) and curr_column == True:
                ### Otherwise case (11,12,13,14,15,16,17,18,10,19,20,21)
                df.loc[~df[curr_column].isin(lst_stack), (Arr1[i])] = Arr2[i]
            else:
                pass

    else:
        if Arr1[i] in list(df.columns):
            # if Arr5[i] == None or Arr5[i] == 'None':
            df.loc[df[Arr1[i]] == 0, Arr1[i]] = Arr2[i]

    # print(lst_stack)


In [15]:
df

,SEC,ST,FSU,IDNO,SEG,SSS,SSU,STRM,SSTRM,SS,...,PS_SS,CWS,ERN_SELF,ERN_REG,newVar1,newVar4,newVar3,newVar6,newVar2,newVar5
0,1,28,17469,1,1,1,1,1,1,1,...,51,51,0,0,ERN_REG,1,ERN_REG,1,ERN_REG,1
1,1,28,17469,1,1,1,1,1,1,1,...,91,91,0,0,ERN_REG,1,ERN_REG,1,ERN_REG,1
2,1,28,17469,1,1,1,1,1,1,1,...,91,91,0,0,ERN_REG,1,ERN_REG,1,ERN_REG,1
3,1,28,17469,1,1,1,1,1,1,1,...,51,51,0,0,ERN_REG,1,ERN_REG,1,ERN_REG,1
4,1,28,17469,2,1,1,2,1,1,1,...,51,51,0,0,ERN_REG,1,ERN_REG,1,ERN_REG,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109851,2,34,27428,7,1,4,1,1,3,2,...,94,94,0,0,ERN_REG,1,ERN_REG,1,ERN_REG,1
109852,2,34,27428,8,1,4,2,1,3,2,...,51,51,0,0,ERN_REG,1,ERN_REG,1,ERN_REG,1
109853,2,34,27428,8,1,4,2,1,3,2,...,91,91,0,0,ERN_REG,1,ERN_REG,1,ERN_REG,1
109854,2,34,27428,8,1,4,2,1,3,2,...,92,92,0,0,ERN_REG,1,ERN_REG,1,ERN_REG,1


In [16]:
# df.to_csv('test.csv')

In [17]:
# Arr5, Arr4, Arr3, Arr2, Arr1

Arr4, Arr3, Arr1, Arr5

([None, None, None, None, None, None, None, None, None, None, None],
 ['CWS', 'CWS', 'CWS', 'CWS', 'CWS', '+', 'None', '+', 'None', '+', 'None'],
 ['newVar4',
  'newVar4',
  'newVar5',
  'newVar5',
  'newVar6',
  'newVar1',
  'newVar1',
  'newVar2',
  'newVar2',
  'newVar3',
  'newVar3'],
 [None, None, None, None, None, None, None, None, None, None, None])

In [18]:
# Check for name mismatches and create new columns
mismatched_names = set(Arr1) - set(df.columns)
new_columns = {name: [0] * df.shape[0] for name in mismatched_names}
df = df.assign(**new_columns)

for i in range(len(Arr1)):
    if Arr3[i] == 'in':
        if isinstance(Arr5[i], int):
            flattened_list = [Arr5[i]]
        else:
            flattened_list = [item for sublist in Arr5[i] for item in sublist] if (isinstance(Arr5[i], list) or isinstance(Arr5[i], tuple)) else [Arr5[i]]
        df.loc[df[Arr4[i]].isin(flattened_list), (Arr1[i])] = Arr2[i]
    elif Arr3[i] == 'NOT in':
        if isinstance(Arr5[i], int):
            flattened_list = [Arr5[i]]
        else:
            flattened_list = [item for sublist in Arr5[i] for item in sublist] if (isinstance(Arr5[i], list) or isinstance(Arr5[i], tuple)) else [Arr5[i]]
        df.loc[~df[Arr4[i]].isin(flattened_list), (Arr1[i])] = 0


In [19]:
df.columns

Index(['SEC', ' ST', 'FSU', ' IDNO', ' SEG', ' SSS', ' SSU', ' STRM',
       '  SSTRM', '  SS', ' NSS', ' NSC', ' MULT', ' HH_SIZE', '  HHTYPE',
       '  RELG', '  SG', ' MHCE', '  DC_ALL', '  DC_ST', '  SRL', ' REL',
       ' SEX', ' MSEX', '  AGE', ' MARST', '  GEDU_LVL', '   PAS', ' PS_SS',
       '  CWS', '  ERN_SELF', '   ERN_REG', 'newVar1', 'newVar4', 'newVar3',
       'newVar6', 'newVar2', 'newVar5'],
      dtype='object')

In [20]:
ldsdsist(set(Arr1)), Arr1, Arr2, Arr4, Arr5

# # check for name mismatches and create new columns
# mismatched_names = set(Arr1) - set(list(df.columns))
# new_columns = {name: [0]*df.shape[0] for name in mismatched_names}
# df = df.assign(**new_columns)
# df.columns

# for k, v in new_columns.items():
#     print(k)
#     df.loc[df[new_columns].isin([5, 6]), ('newVar1')] = df['    ERN_REG'] + df['   ERN_SELF']
#     df.loc[~df['  GEDU_LVL'].isin([5, 6]), ('newVar1')] = 0

NameError: ignored

In [ ]:
print(new_columns)

In [ ]:

# df['newVar1'] = pd.Series([])
# df.loc[df['  GEDU_LVL'].isin([5, 6]), ('newVar1')] = df['    ERN_REG'] + df['   ERN_SELF']
# df.loc[~df['  GEDU_LVL'].isin([5, 6]), ('newVar1')] = 0

In [ ]:
import re

# input string
string = 'NOT in(CWS,"11,12,21,31,41,51,61,61,71,72")'

# pattern
pattern = r'^NOT\s+in\((\w+),\s*"([^"]+)"\)$'

# match
match = re.match(pattern, string)

print(match.group(1), type(match.group(2)))

# extract groups
if match:
    cws = match.group(1)
    integers = match.group(2)
    integers_list = integers.split(',')
    integers_set = set(integers_list)
    print(f"CWS: {cws}, Integers: {integers_set}")
else:
    print("No match found.")


# convert integers to set of integers
integers_set = set(map(int, integers_list))
print(f"CWS: {cws}, Integers: {integers_set}")
integers_set

In [ ]:
### Reading and filtering column names
df = pd.read_csv(FILE_(data))
df = df.rename(columns=lambda x: x.strip())

# print(NEWVARIABLE_(data, df)[0])

for col in RENAME_(data, df):
    col = [elem for elem in col.split(' ') if elem.strip()]
    
    # df[col[0]] = df[]

    col_search = col[2].split('+')

    # print(col_search)

    for txt in col_search:
        if txt in df.columns.str.lower():
            pass
            
        # else:
        #     print('No')

    # print(col[2].split('+'))

# df.query(FILTER_(data))

# df = pd.read_csv(FILE_(data))
# df.head(2)

In [ ]:
# ## read the RSE file and extract the lines from it
# with open('Example1.RSE') as f:

#     ## Reading each line of the file 
#     lines = f.readlines()

# ## To generate keys of the dictionary 
# keys = []

# for line in lines:
#     # print(line)
#     line = line.strip()

#     if line.startswith("#"):

#         quote_index = line.find("'") 
#         if quote_index != -1:
#             key = line[1:quote_index].split()

#             # Check if the line is empty
#             if not line:
#                 # Empty line found, break out of the loop
#                 break

#             if key not in keys:
#                 keys.append(key)

# print(keys)

In [ ]:
### renaming the variables
### **********************

# rename the columns using a dictionary
new_columns = {
    'FSU': 'fsu',
    'NS_SSS': 'NSS'
}

df = df.rename(columns=new_columns)

df['STRMID'] = df['SEC'].apply(lambda x: f'0{x}' if x < 10 else str(x)) + \
                df['AST'].apply(lambda x: f'0{x}' if x < 10 else str(x)) + \
                 df[' STRM'].apply(lambda x: f'0{x}' if x < 10 else str(x)) + \
                  df['  SSTRM'].apply(lambda x: f'0{x}' if x < 10 else str(x))

df[' SSU'] = df[' FSU'].astype(str) + df[' IDNO'].apply(lambda x: f'0{x}' if x < 10 else str(x))

In [ ]:
df[' SSU']

In [ ]:
age = df['  AGE']

# Function to calculate the right age group
def get_age_group(age):
    if age < 5:
        return "01"
    elif age >= 5 and age < 10:
        return "02"
    elif age >= 10 and age < 20:
        return "03"
    elif age >= 20 and age < 60:
        return "04"
    elif age >= 60:
        return "05"
    else:
        return "00"

# Apply the get_age_group function to each row in the "age" column in parallel
df['AGE_GROUP'] = df['  AGE'].parallel_apply(get_age_group)


In [ ]:
### Age group with right age
def get_result(df):
    age = df['  AGE']
    ageGrp = df['AGE_GROUP']

    if ageGrp == "01" and age < 5:
        return "01"
    elif ageGrp == "02" and age >= 5 and age < 10:
        return "02"
    elif ageGrp == "03" and age >= 10 and age < 20:
        return "03"
    elif ageGrp == "04" and age >= 20 and age < 60:
        return "04"
    elif ageGrp == "05" and age>=60:
        return "00"
    else:
        return "00"

df['AGE_GROUP'] = df.parallel_apply(get_result, axis=1)

In [ ]:
df.columns

In [ ]:
def Y1_(df):
    df['newVar1'] = pd.Series([])
    df.loc[df['  GEDU_LVL'].isin([5, 6]), ('newVar1')] = df['    ERN_REG'] + df['   ERN_SELF']
    df.loc[~df['  GEDU_LVL'].isin([5, 6]), ('newVar1')] = 0

def Y2_(df):
    df['newVar2'] = pd.Series([])
    df.loc[df['  GEDU_LVL'].isin([7, 8, 9]), ('newVar2')] = df['    ERN_REG'] + df['   ERN_SELF']
    df.loc[~df['  GEDU_LVL'].isin([7, 8, 9]), ('newVar2')] = 0

def Y3_(df):
    df['newVar3'] = pd.Series([])
    df.loc[df['  GEDU_LVL'].isin([5, 6, 7, 8, 9]), ('newVar3')] = df['    ERN_REG'] + df['   ERN_SELF']
    df.loc[~df['  GEDU_LVL'].isin([5, 6, 7, 8, 9]), ('newVar3')] = 0

def Y4_(df):
    df['newVar4'] = pd.Series([])
    df.loc[df['   ACWS'].isin([11,12,21,31,41,51,61,61,71,72]), ('newVar4')] = 1
    df.loc[~df['   ACWS'].isin([11,12,21,31,41,51,61,61,71,72]), ('newVar4')] = 0

def Y5_(df):
    df['newVar5'] = pd.Series([])
    df.loc[df['   ACWS'].isin([11,12,21,31,41,51,61,61,71,72,81]), ('newVar5')] = 1
    df.loc[~df['   ACWS'].isin([11,12,21,31,41,51,61,61,71,72,81]), ('newVar5')] = 0

def Y6_(df):
    df['newVar6'] = 1

In [ ]:
Y1_(df), Y2_(df), Y3_(df), Y4_(df), Y5_(df), Y6_(df)

In [ ]:
# ### Example-2: Defining another variable

# print("select number of variables : ")

# n_var = int(input())

# ### creating no. of n_var
# for i in range(1, n_var+1):
#     # df['Y' + str(i)] = pd.Series([])
#     df['X' + str(i)] = pd.Series([])

#     if i == 1:
#         df.loc[df['  GEDU_LVL'].isin([5, 6]), ('X' + str(i))] = df['    ERN_REG'] + df['   ERN_SELF']
#         df.loc[~df['  GEDU_LVL'].isin([5, 6]), ('X' + str(i))] = 0
#     elif i == 2:
#         df.loc[df['  GEDU_LVL'].isin([7, 8, 9]), ('X' + str(i))] = df['    ERN_REG'] + df['   ERN_SELF']
#         df.loc[~df['  GEDU_LVL'].isin([5, 6]), ('X' + str(i))] = 0
#     elif i == 3:
#         df.loc[df['  GEDU_LVL'].isin([5, 6, 7, 8, 9]), ('X' + str(i))] = df['    ERN_REG'] + df['   ERN_SELF']
#         df.loc[~df['  GEDU_LVL'].isin([5, 6]), ('X' + str(i))] = 0
    

In [ ]:
df['newVar3'].unique()

In [ ]:
### Transform the variables



In [ ]:
df.to_csv('GROUP SEC ST AGE_Group.csv')

In [ ]:
df